In [43]:
from google.colab import drive
drive.mount('/content/drive')
import os
import torch
import random

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
path = '/content/drive/My Drive/social_computing/data/count.txt'

In [31]:
f = open(path, encoding = "utf-8")
file_data = f.readlines()

In [32]:
from typing import Dict
id_list = []
for line in file_data:
  id_list.append(line.strip('\n').split(' ')[3][:-1])

# data_list['id'] = [PullRequest_count, Fork_count, Watch_count, IssueComment_count, class]
map = dict()
map['PullRequestEvent'] = 0
map['ForkEvent'] = 1
map['WatchEvent'] = 2
map['IssueCommentEvent'] = 3
data_list = dict()

In [33]:
# Initialize data_list
for id in id_list:
  data_list[id] = [0,0,0,0,0]

In [34]:
for line in file_data: 
  raw = line.strip('\n').split(' ') 
  data_list[raw[3][:-1]][map[raw[1][:-1]]] = int(raw[-1])

In [38]:
p_list = []
final_data = []
for id in id_list:
  p_list.append(data_list[id][0])
p_list.sort(reverse=True)
threshold = p_list[int(len(p_list)*0.1)]
for id in id_list:
  if data_list[id][0]>=threshold:
    data_list[id][4] = 1
  final_data.append(data_list[id][1:])

In [39]:
final_data = torch.tensor(final_data)

In [40]:
print(final_data.shape)

torch.Size([883436, 4])


In [68]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.labels = data[:, -1]
        self.x = data[:, 0:-1]
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.x[idx].float()
        y = self.labels[idx]
        return x, y

In [69]:
# randomly sample 70% as training data, and remaining 30% data as testing data
sample = [i for i in range(len(final_data))]
sample = random.sample(sample, len(final_data))
train = Dataset(final_data[sample[:int(len(final_data)*0.7)]])
train_dataloader = torch.utils.data.DataLoader(train, batch_size=256, shuffle=True, drop_last=True)

In [70]:
from torch.optim import Adam
from tqdm import tqdm
from torch import nn
def train(model, trainloader, learning_rate, epochs):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    for epoch_num in range(epochs):
      acc_train = 0
      loss_train = 0
      for train_input, train_label in tqdm(trainloader):
        model.zero_grad()
        train_input = train_input.to(device)
        train_label = train_label.to(device)
        output = model(train_input)
        batch_loss = criterion(output, train_label)
        loss_train += batch_loss.item()
        batch_loss.backward()
        optimizer.step()
        with torch.no_grad():
          label_new = output.argmax(dim=1)
          acc_train = acc_train + (train_label==label_new).float().mean()
      with torch.no_grad():
        loss_train = loss_train / len(trainloader)
        acc_train = acc_train / len(trainloader)
        print(f'training loss:{loss_train}, training accuracy:{acc_train}')

In [78]:
model = nn.Sequential(nn.Linear(3,2))
train(model, trainloader=train_dataloader, learning_rate=11, epochs=20)

100%|██████████| 4831/4831 [00:10<00:00, 465.96it/s]


training loss:68.62970850901237, training accuracy:0.8554679155349731


 64%|██████▍   | 3091/4831 [00:06<00:03, 442.70it/s]


KeyboardInterrupt: ignored

In [76]:
test_data = final_data[sample[int(len(final_data)*0.7):]][:, :-1].float()
test_label = final_data[sample[int(len(final_data)*0.7):]][:, -1].long()
(model(test_data).argmax(dim=1)==test_label).float().mean()

tensor(0.9002)